<a id='top'></a>
# Imaging Mode Data Calibration: Part 2 - Calibrated Slope Images
---
**Author**: Bryan Hilbert (hilbert@stsci.edu)| **Latest Update**: October 4, 2022



<div class="alert alert-block alert-warning">
    <h3><u><b>Notebook Goals</b></u></h3>
    <ul>Working with the Stage 2 Calibration Pipeline, we will:</ul>    
<ul>
    <li>Look at the different ways to call the pipeline</li>
    <li>Examine exactly what each pipeline step does to the science data</li>    
</ul>
</div>

## Table of Contents
* [Introduction](#intro)
* [Pipeline Resources and Documentation](#resources)
   * [Installation](#installation)
   * [Reference Files](#reference_files)
* [Imports](#Imports_ID)
* [Convenience Functions](#convenience_functions)
* [Download Data](#download_data)
* [Association Files](#associations)
* [Methods for calling steps/pipelines](#calling_methods)
* [calwebb_image2 - Calibrated slope images](#image2) 
   * [Run the entire pipeline](#image2_at_once)
       * [Using the run() method](#run_method)
       * [Using the call() method](#call_method)
       * [Using the command line](#command_line)
       * [Exercise: run Image2 pipeline on MIRI data](#exercise)
   * [Run the individual pipeline steps](#image2_step_by_step)
       * [The `WCS Creation` step](#assign_wcs)
       * [The `Flat Fielding` step](#flatfield)
       * [The `Photometric calibration` step](#photom)
       * [The `Resample` step](#resample)
* [Exercise Solution](#exercise_solution)

<a id='intro'></a>
## Introduction

This notebook covers part 2 of the imaging mode data calibration module. In this notebook we'll review Stage 2 of the JWST calibration pipeline for imaging data, also known as *calwebb\_image2*. 

The [Stage 2 pipeline](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_image2.html) applies instrumental corrections and calibrations to the slope images output from Stage 1. This includes background subtraction, the creation of a full World Coordinate System (WCS) for the data, application of the flat field, and flux calibration. In most cases the final output is an image in units of surface brightness. Whereas the input files had suffixes of `*_rate.fits*`, the output files have suffixes of `*_cal.fits*`.

In addition to the steps above, the Stage 2 pipeline will also run the [resample](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/main.html) step on the calibrated images, in order to remove the effects of instrument distortion. This step outputs files with the suffix `*_i2d.fits*` that contain "rectified" images. However, these files are meant only for user examination of the data. It is the `*_cal.fits*` files that are passed on to Stage 3 of the pipeline.

To illustrate how the steps of the pipeline change the input data, we will download several sample files and run them through the pipeline, examining the results at several places along the way.

All JWST imaging mode data, regardless of instrument, are processed through the *calwebb\_image2* pipeline. The steps and the order in which they are performed is the same for all data. For the purposes of this notebook, we will continue with the processing of the NIRCam data used in the Stage 1 notebook. We will also provide example MIRI files that can be used in a separate exercise.

<a id='resources'></a>
## Pipeline Resources and Documentation

There are several different places to find information on installing and running the pipeline. This notebook will give a shortened description of the steps pulled from the detailed pipeline information pages, but to find more in-depth instructions use the links below.

* [JWST Documentation (JDox) for the Stage 2 pipeline](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline-overview/stages-of-jwst-data-processing/calwebb_image2) including short a short summary of what each step does.

* [High-level description of all pipeline stages and steps](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html)

* [`jwst` package documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html) including how to run the pipeline, input/output files, etc.

* [`jwst` package GitHub repository, with installation instructions](https://github.com/spacetelescope/jwst/blob/master/README.md)

* [**Help Desk**](https://stsci.service-now.com/jwst?id=sc_cat_item&sys_id=27a8af2fdbf2220033b55dd5ce9619cd&sysparm_category=e15706fc0a0a0aa7007fc21e1ab70c2f): **If you have any questions or problems regarding the pipeline, submit a ticket to the Help Desk**

<a id='installation'></a>
### Installation

<div class="alert alert-block alert-info">
    During the JWebbinar, we will be working in a pre-existing environment where the <b>jwst</b> package has already been installed, so you won't need to install it yourself.
</div>

<div class="alert alert-block alert-warning">
If you wish to run this notebook outside of this JWebbinar, you will have to first install the <b>jwst</b> package.<br>

For more detailed instructions on the various ways to install the package, see the [installation instructions](https://github.com/spacetelescope/jwst/blob/master/README.md) on GitHub.

The easiest way to install the pipeline is via `pip`. Below we show how to create a new conda environment, activate that environment, and then install the latest released version of the pipeline. You can name your environment anything you like. In the lines below, replace `<env_name>` with your chosen environment name.

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install jwst`

If you wish to install the development version of the pipeline, which is more recent than (but not as well tested compared to) the latest released version:

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install git+https://github.com/spacetelescope/jwst`
    
</div>

<a id='reference_files'></a>
### Reference Files

[Calibration reference files](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline-overview/jwst-data-calibration-reference-files) are a collection of FITS and ASDF files that are used to remove instrumental signatures and calibrate JWST data. For example, the dark current reference file contains a multiaccum ramp of dark current signal to be subtracted from the data during the dark current subtraction step. 

When running a pipeline or pipeline step, the pipeline will automatically look for any required reference files in a pre-defined local directory. If the required reference files are not present, they will automatically be downloaded from the Calibration Reference Data System (CRDS) at STScI.

<div class="alert alert-block alert-info">
    During the JWebbinar, our pre-existing existing environment is set up to correctly use and store calibration reference files, and you do not need to set the environment variables below.
</div>
    
<div class="alert alert-block alert-warning">
If you wish to run this notebook outside of this JWebbinar, you will have to specify a local directory in which to store reference files, along with the server to use to download the reference files from CRDS. To accomplish this, there are two environment variables that should be set prior to calling the pipeline. These are the CRDS_PATH and CRDS_SERVER_URL variables. In the example below, reference files will be downloaded to the "crds_cache" directory under the home directory.

>`$ export CRDS_PATH=$HOME/crds_cache`<br>
>`$ export CRDS_SERVER_URL=https://jwst-crds-pub.stsci.edu`<br>
OR:<br>
`os.environ["CRDS_PATH"] = "/user/myself/crds_cache"`<br>
`os.environ["CRDS_SERVER_URL"] = "https://jwst-crds-pub.stsci.edu"`<br>

The first time you run the pipeline, the [CRDS server](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html#crds) should download all of the context and reference files that are needed for that pipeline run, and dump them into the CRDS_PATH directory. Subsequent executions of the pipeline will first look to see if it has what it needs in CRDS_PATH and anything it doesn't have will be downloaded from the STScI cache. 
</div>

<div class="alert alert-block alert-warning">NOTE: Users at STScI should automatically have access to the Calibration Reference Data System (CRDS) cache for running the pipeline, and can skip setting these environment variables.</div>

[Top of Notebook](#top)

<a id=#Imports_ID></a>
## Imports

Import packages necessary for this notebook

In [ ]:
# Packages that allow us to get information about objects:
from glob import glob
#Modify the path to a directory on your machine
import os
os.environ["CRDS_PATH"] = "/Users/cpacifici/crds_cache_new/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds-pub.stsci.edu"
import shutil

# Numpy library:
import numpy as np

# To read association file
import json

# To download data
import requests

# To examine parameter reference files
import asdf

# Astropy tools:
from astropy.io import fits
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch

Set up matplotlib for plotting

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# Use this version (outside of Jupyter Lab) if you want interactive plots
#%matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

Import JWST pipeline-related modules

In [ ]:
# The entire calwebb_image2 pipeline
from jwst.pipeline import calwebb_image2

# Individual steps that make up calwebb_image2
from jwst.background import BackgroundStep
from jwst.assign_wcs import AssignWcsStep
from jwst.flatfield import FlatFieldStep
from jwst.photom import PhotomStep
from jwst.resample import ResampleStep
from jwst import datamodels

Check which version of the pipeline we are running:

In [ ]:
import jwst
print(jwst.__version__)

<a id='convenience_functions'></a>
## Define convenience functions and parameters

Here we define some functions that we will use repeatedly throughout the notebook.

In [ ]:
# Files created in this notebook will be saved
# in the current working directory
output_dir = './'

In [ ]:
def download_files(files, output_directory, force=False):
    """Given a tuple or list of tuples containing (URL, filename),
    download the given files into the current working directory.
    Downloading is done via astropy's download_file. A symbolic link
    is created in the specified output dirctory that points to the
    downloaded file.
    
    Parameters
    ----------
    files : tuple or list of tuples
        Each 2-tuple should contain (URL, filename), where
        URL is the URL from which to download the file, and
        filename will be the name of the symlink pointing to
        the downloaded file.
        
    output_directory : str
        Name of the directory in which to create the symbolic
        links to the downloaded files
        
    force : bool
        If True, the file will be downloaded regarless of whether
        it is already present or not.
        
    Returns
    -------
    filenames : list
        List of filenames corresponding to the symbolic links
        of the downloaded files
    """
    # In the case of a single input tuple, make it a
    # 1 element list, for consistency.
    filenames = []
    if isinstance(files, tuple):
        files = [files]
        
    for file in files:
        filenames.append(file[1])
        if force:
            print('Downloading {}...'.format(file[1]))
            demo_file = download_file(file[0], cache='update')
            # Make a symbolic link using a local name for convenience
            if not os.path.islink(os.path.join(output_directory, file[1])):
                os.symlink(demo_file, os.path.join(output_directory, file[1]))
        else:
            if not os.path.isfile(os.path.join(output_directory, file[1])):
                print('Downloading {}...'.format(file[1]))
                demo_file = download_file(file[0], cache='update')
                # Make a symbolic link using a local name for convenience
                os.symlink(demo_file, os.path.join(output_directory, file[1]))
            else:
                print('{} already exists, skipping download...'.format(file[1]))
                continue
    return filenames    

In [ ]:
def show_image(data_2d, vmin, vmax, xpixel=None, ypixel=None, title=None,
               scale='log', units='MJy/str'):
    """Function to generate a 2D, log-scaled image of the data, 
    with an option to highlight a specific pixel.
    
    Parameters
    ----------
    data_2d : numpy.ndarray
        2D image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    xpixel : int
        X-coordinate of pixel to highlight
        
    ypixel : int
        Y-coordinate of pixel to highlight
        
    title : str
        String to use for the plot title
        
    scale : str
        Specify scaling of the image. Can be 'log' or 'linear'
        
    units : str
        Units of the data. Used for the annotation in the
        color bar
    """
    if scale == 'log':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LogStretch())
    elif scale == 'linear':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LinearStretch())
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm)
    
    if xpixel and ypixel:
        plt.plot(xpixel, ypixel, marker='o', color='red', label='Selected Pixel')

    fig.colorbar(im, label=units)
    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    if title:
        plt.title(title)

[Top of Notebook](#top)

<a id='download_data'></a>
## Download Data

For this module, we will use rate files from a NIRCam imaging exposure that is stored in Box. Let's grab them:

Download the rate files, association file, and parameter reference file, so that we have inputs to work with.

In [ ]:
nircam_info = [('https://stsci.box.com/shared/static/6v78it78ylm6xcu7ud1x2k8xmas0bzcc.fits',
                'jw02727002001_02105_00001_nrcblong_rate.fits'),
               ('https://stsci.box.com/shared/static/uyr1yw2ox9l8yt18iuuo5q6omd3ybv2m.fits',
                'jw02727002001_02105_00002_nrcblong_rate.fits'),
               ('https://stsci.box.com/shared/static/fks19e019xul3em5my29top6kbif2893.fits',
                'jw02727002001_02105_00003_nrcblong_rate.fits'),
               ('https://stsci.box.com/shared/static/9mxdzg0wt02kqtwx7j8h4f15hlanlxal.json',
                'jw02727-o002_20220802t161539_image2_asn.json'),
               ('https://stsci.box.com/shared/static/d4pu8ieyjc27wzoe0of3ajb9vjtvc80g.asdf',
                'image2_pipeline_params.asdf')]
nircam_files = download_files(nircam_info, output_dir, force=False)

<a id='associations'></a>
## Association Files

The Stage 2 pipeline can be called on a single fits file, or a collection of fits files. When calling on multiple files, the input is a json-formatted file called an ["association" file](https://jwst-pipeline.readthedocs.io/en/stable/jwst/associations/index.html) that lists each of the fits files to be processed. 

* An association is a means of identifying a set of exposures that belong together and may be dependent upon one another.
* The association concept permits exposures to be calibrated, archived, retrieved, and reprocessed as a set rather than as individual objects.

When retrieving your observations from MAST, you will be able to download the association files for your data along with the fits files containing the observations.

The association file presents your data files in organized groups. Let's open the level 2 association file for the data to be processed in this notebook and look at its contents.

In [ ]:
# Open the file and load into json object
asn_file = os.path.join(output_dir, 'jw02727-o002_20220802t161539_image2_asn.json')

In [ ]:
with open(asn_file) as f_obj:
    asn_data = json.load(f_obj)

In [ ]:
asn_data

Here we see that the association file begins with a few lines of data that give high-level information about the association. The most important entry here is the `asn_rule` field. Association files have different formats for the different stages of the pipeline. You should be sure that the `asn_rule` matches the pipeline that you will be running. In this case we'll be running the Stage 2 pipeline, and we see that the `asn_rule` mentions "Level2b", which is what we want.

Beneath these lines, we see the `products` field. This field contains a list of dictionaries that specify the files that belong to this association, and the types of those files. When the Stage 2 pipeline is run on this association file, all files listed here will be run through the calibration steps.

<a id='calling_methods'></a>
## Methods for calling steps/pipelines

There are three common methods by which the pipeline or pipeline steps can be called. From within python, the `run()` and `call()` methods of the pipeline or step classes can be used. Alternatively, the `strun` command can be used from the command line. Within this notebook, in the section where we [call the entire pipeline](#image2_at_once), we show examples of all three methods. For the remainder of the pipeline steps, we will focus on using the `run()` method.

When using the `call()` method or `strun`, optional input parameters can be specified via [parameter reference files](#parameter_reffiles). When using the `run()` method, these parameters are instead specified within python.

<a id='parameter_reffiles'></a>
## Parameter Reference Files

When calling a pipeline or pipeline step using the `call()` method or the command line, [parameter reference files](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/config_asdf.html#config-asdf-files) can be used to specify values for input parameters. These reference files are [asdf](https://asdf.readthedocs.io/en/stable/) format and appear somewhat similar to json files when examined in a text editor. 

Versions of parameter reference files containing default parameter values for each step and pipeline are available in CRDS. When using the `call()` method, if you do not specify a parameter reference file name in the call, the pipeline or step will retrieve and use the appropriate file from CRDS, which will then run the pipeline or step with the parameter values in that file. If you provide the name of a parameter reference file, then the parameter values in that file will take precedence. For any parameter not specified in your parameter reference file, the pipeline will use the default value.

When using `strun`, the parameter reference file is a required input.

Let's take a look at the contents of a parameter reference file. We'll open it using the asdf package, and use the tree attribute to see what's inside:

In [ ]:
image2_param_reffile = os.path.join(output_dir, 'image2_pipeline_params.asdf')

In [ ]:
im2_reffile = asdf.open(image2_param_reffile)

In [ ]:
im2_reffile.tree

The top part of the file contains various metadata entries about the file itself. Below that, you'll see a `'name'` entry, which lists `Image2Pipeline` as the class to which these parameters apply. The next line contains the `parameters` entry, which lists parameters and values attached to the pipeline itself. Below this is the `steps` entry, which contains a list of dictionaries. Each dictionary refers to one step within the pipeline, and specifies parameters and values that apply to that step. If you look through these entries, you'll see the same parameters and values that we specified manually when using the `run()` method above.

In [ ]:
# Don't forget to close the file
im2_reffile.close()

[Top of notebook](#top)

---
<a id='image2'></a>
## The calwebb_image2 pipeline: Calibrated slope images

In the sections below, we will run the Stage 2 pipeline using an association file containing several NIRCam exposures. We will first call the entire *calwebb_image2* pipeline itself. The pipeline is a wrapper which will string together all of the appropriate steps in the proper order. The final outputs from this call are a calibrated slope image which is ready to go into the Stage 3 pipeline (with a suffix of `_cal.fits`), as well as a calibrated slope image which has been resampled in order to remove distortion effects (with a suffix of `_i2d.fits`). The latter is only for user-examination. The `_cal.fits` file is used as input to the Stage 3 pipeline. Note that the units in these output images are now physical units (MJy/str), rather than DN/sec.

After running the entire pipeline, we will go back to the original uncalibrated slope images and manually run them through each of the steps that comprise the Stage 2 pipeline. For each step we will describe in more detail what is going on and examine how the exposure files have changed.

See [Figure 1](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline-overview/stages-of-jwst-data-processing/calwebb_image2) on the calwebb_image2 algorithm page for a map of the steps are performed on the input data.

<a id='image2_at_once'></a>
### Run the entire `calwebb_image2` pipeline

In this section we show how to run the entire calwebb_image2 pipeline with a single call. 

We set parameter values for some of the individual steps, save some outputs, etc, and then call the pipeline.

We will call the pipeline using the `run()` method and while that is running, we will go over the equivalent `call()` and `strun` commands, examine some of the pipeline log entries that are printed to the screen, and then look at the pipeline output.

<a id='run_method'></a>
#### Call the pipeline using the run() method

When using the `run()` method to execute a pipeline (or step), the pipeline class is first instantiated without the data to be processed. Optional input parameters are specified using attributes of the class instance. Finally, the call to the `run()` method is made and the data are supplied.  See here for [more examples of the run() method](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/call_via_run.html).

The `run()` method does not take any kind of parameter reference file as input. If you wish to set values for various parameters, you must do that manually. Below, we set several paramaters in order to show how it's done. 

How do you know what parameters are available to be set and what their default values are? The `spec` property for individual steps will list them. The property is less useful for the pipelines themselves, as it does not show the parameters for the steps compirising the pipeline.

All steps and pipelines have several common parameters that can be set. 

* `save_results` specifies whether or not to save the output of that step/pipeline to a file. The default is False.
* `output_dir` is the directory into which the output files will be saved.
* `output_file` is the base filename to use for the saved result. Note that each step/pipeline will add a custom suffix onto output_file. 

Let's look at the available parameters for the resample step, and manually set some of these in our call to `run()`.

In [ ]:
print(ResampleStep.spec)

<a id='detector1_using_run'></a>

<div class="alert alert-block alert-info">
Finally, let's run the pipeline. The output can be a little overwhelming. There will be multiple log entries printed to the screen for each step.
</div>

In [ ]:
# Create an instance of the pipeline class
image2 = calwebb_image2.Image2Pipeline()

# Set some parameters that pertain to the
# entire pipeline
image2.output_dir = output_dir
image2.save_results = True

# Set some parameters that pertain to some of
# the individual steps
image2.resample.pixfrac = 1.0    # this is the default. Set here as an example

# Call the run() method
image2.run(asn_file)

This cell will take a few seconds to run. While we're waiting, let's look at the other two methods that can be used to call the pipeline. Then we'll come back here and look at the log meessages output by this cell so we can see what happened.

<a id='call_method'></a>
#### Call the pipeline using the call() method

When using the `call()` method, a single command will instantiate and run the pipeline (or step). The input data and optional parameter reference files are supplied in this single command. In this case, any desired input parameters cannot be set after instantiation, as with the `run()` method. See here for [example usage of call() method](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/call_via_call.html).

The commands below will call the pipeline using the `call()` method and will supply the parameter reference file. Since we just ran the pipeline with the `run()` method above, we won't actually execute the call to `call()`. But if you wish to try it out, use the pull-down menu above to change the cell to be 'Code', and then execute it. (Or, Click 'Cell' > 'Cell Type' > 'Code')

<div class="alert alert-block alert-info">

<b>Method #1:</b>
Provide the name of the observation file, the pipeline-specific input paramters, and the name of the parameter reference file that specifies step-specific parameters
</div>

<div class="alert alert-block alert-info">

<b>Method #2:</b>
In this case, build a nested dictionary that specifies parameter values for various steps, and provide it in the call to call().
</div>

<a id='command_line'></a>
#### Call the pipeline from the command line

Calling a pipeline or step from the command line is similar to using the `call()` method. The data file to be processed, along with an optional parameter reference file and optional parameter/value pairs can be provided to the `strun` command. See here for [additional examples of command line calls](https://jwst-pipeline.readthedocs.io/en/stable/jwst/introduction.html?highlight=%22command%20line%22#running-from-the-command-line).

The cells below contains two different command line commands that use `strun` to call the calwebb_image2 pipeline. 

<div class="alert alert-block alert-info">

<b>Method #1:</b>
We provide the name of the pipeline class, the association file, and explicitly set pipeline- and step-specific parameters. 
    
```
    strun jwst.pipeline.Image2Pipeline level2_lw_asn.json --steps.resample.pixfrac=1.0
```
</div>

<div class="alert alert-block alert-info">

<b>Method #2:</b>
This version of the command is much more succinct, as the parameter values to be set are all contained within the parameter reference file. The pipeline class is also contained in the parameter reference file, so there is no need to specify it in the command itself.
    
```
    strun image2_pipeline_params.asdf level2_lw_asn.json
```
</div>

### Examine the outputs

Get the filenames from the association file

In [ ]:
# Get a list of input file names from the association file
input_files = [item['members'][0]['expname'] for item in asn_data['products']]       

In [ ]:
input_files

In [ ]:
# Get a list of the output file names
output_files = sorted(glob(os.path.join(output_dir, 'jw02727*_cal.fits')))

In [ ]:
output_files

In [ ]:
# Open the first calibrated output file
cal_data = fits.open(output_files[0])

In [ ]:
# Check the contents of the calibrated file
cal_data.info()

Let's look at the header of the SCI extension, to see the information that has been added by the assign WCS and flux calibration steps.

In [ ]:
cal_data['SCI'].header

In [ ]:
# Get the name of the `i2d` file associated with the first output file
i2d_file = output_files[0].replace('cal.fits', 'i2d.fits')

In [ ]:
# Extract the data from the i2d file so we can look at it.
i2d_data = fits.getdata(i2d_file)

<a id='pipeline_output_view'></a>
Display the calibrated slope image and the distortion-free output file

In [ ]:
show_image(cal_data['SCI'].data, 0.3, 5)

In [ ]:
show_image(i2d_data, 0.3, 5)

[Top of Notebook](#top)

<a id='image2_step_by_step'></a>
## Run the individual pipeline steps

In the sections below we run the steps contained within calwebb_image2 one at a time, in order to more clearly see what each step is doing.

<a id='assign_wcs'></a>
### The `WCS creation` step

#### Summary

This step adds a World Coordinate System (WCS) object to the observation. The WCS object contains transformations between positions on the detector to positions in a world coordinate frame.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/assign_wcs/main.html) of the step.

#### Arguments

There are no optional arguments for this step.

#### Reference files used

The [reference files used](https://jwst-pipeline.readthedocs.io/en/stable/jwst/assign_wcs/reference_files.html) in this step depend on the instrument used. The primary reference file used is the `DISTORTION` reference file, which contains coefficients that can be used to translate between various coordinate systems.

#### Run the step

The assign_wcs step expects an instance of an ImageModel as input, rather than an association file or fits file. So in this case we'll loop over the input files, read them into ImageModel instances, and call the step. Results will be saved to fits files.

In [ ]:
for filename in input_files:
    image = datamodels.ImageModel(filename)
    
    assign_wcs_step = AssignWcsStep()
    assign_wcs_step.output_dir = output_dir
    assign_wcs_step.save_results = True
    assign_wcs_step.run(filename)

In [ ]:
# When the output is saved, the assign_wcs step will
# attach a suffix of 'assignwcsstep' to the input filename.
assign_wcs_output_files = sorted(glob(os.path.join(output_dir, '*_assignwcsstep.fits')))

Let's look into the WCS information that this step added to the files

In [ ]:
# Open the output file using datamodels, so we have access to the full WCS
model = datamodels.ImageModel(assign_wcs_output_files[0])

The full GWCS model is contained in the ASDF extension of the file, and can be seen through the `meta` property. 

In [ ]:
# Look at the WCS info in the calibrated image model 
model.meta.wcs

There are several world coordinate systems available in the file. 

In [ ]:
# What coordinate frames are available?
model.meta.wcs.available_frames

In [ ]:
# What is the input frame of the WCS object?
model.meta.wcs.input_frame

In [ ]:
# What is the output frame of the WCS object?
model.meta.wcs.output_frame

Let's create a transformation function to go from detector pixels to location on the sky.

In [ ]:
# Get the transform to go from detector to world coordinates
detector_to_world = model.meta.wcs.get_transform('detector', 'world')

And a function for the inverse transformation

In [ ]:
world_to_detector = model.meta.wcs.get_transform('world', 'detector')

Let's look at the calwebb_image2 pipeline output image from before (the _cal.fits files), and zoom in on an interesting area

In [ ]:
show_image(cal_data[1].data[1650:1850, 1150:1350] , 0.35, 10)

In [ ]:
# Close the file
cal_data.close()

Using the transformation functions we defined above, we can now easily determine the RA and Dec of some randomly-chosen sources. Pick out the (x, y) values for a trio of sources above. Remember to add the offset corresponding to the lower left corner of our zoomed in aperture.

In [ ]:
x_offset = 1150
y_offset = 1650
sources_x = np.array([25, 70, 110]) + x_offset
sources_y = np.array([65, 50, 50]) + y_offset

In [ ]:
# Call the transform function
sources_ra, sources_dec = detector_to_world(sources_x, sources_y)

In [ ]:
sources_ra

In [ ]:
sources_dec

And now the opposite case: My target is at a given RA and Dec, so where is it in this image?

In [ ]:
targ_ra = 9.40656553
targ_dec = -33.67799721

In [ ]:
# Call the inverse transform function
targ_x, targ_y = world_to_detector(targ_ra, targ_dec)

In [ ]:
print('Target located at (x, y) = ({}, {})'.format(targ_x, targ_y))

<a id='flatfield'></a>
## The `Flat Fielding` step

#### Summary

This step divides the data by a flat field in order to correct for pixel-to-pixel sensitivity variations.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/flatfield/main.html) of the step.

#### Arguments

There is a [single optional argument](https://jwst-pipeline.readthedocs.io/en/stable/jwst/flatfield/arguments.html) for this step, which applies only to NIRSpec data.

#### Reference files used

This step uses the [`FLAT`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/flatfield/reference_files.html) reference file. 

#### Run the step

In this and the subsequent steps, we will loop over the files output by the prceding step and run the step. 

Why not use an association file as input? Because we would need a separate association file for each step since the filenames to be used as input are different in each step. So in order to avoid dealing with many association files, we simply loop over the filenames.

In [ ]:
for filename in assign_wcs_output_files:
    flatfield_step = FlatFieldStep()
    flatfield_step.output_dir = output_dir
    flatfield_step.save_results = True

    flatfield_step.run(filename)

In [ ]:
# When the output is saved, the flat field step will
# attach a suffix of 'flatfieldstep' to the input filename.
flatfield_output_files = sorted(glob(os.path.join(output_dir, '*_flatfieldstep.fits')))

In [ ]:
before_flat = fits.getdata(assign_wcs_output_files[-1])
after_flat = fits.getdata(flatfield_output_files[-1])

# Some pixels were saturated in all groups of the integration.
# This caused them to have a value of 0.0 in the slope image.
# For this display, let's set those pixels equal to 1.0, just
# to get a clearer picture.
zeros = after_flat == 0
before_flat[zeros] = 1.0
after_flat[zeros] = 1.0

# Recover the flat by taking the ratio of the data before and after
# the flat field step
flat_ratio = before_flat / after_flat

In [ ]:
show_image(after_flat , 0.95, 10)

Taking the ratio of the data before and after the flat field step, we recover the flat field.

In [ ]:
show_image(flat_ratio, 0.9, 1.1, scale='linear', units='Flat Field Value')

<a id='photom'> </a>
## The `Photometric calibration` step

#### Summary

This step applies flux (photometric) calibration to the data, converting it from units of ADU/sec to surface brightness. A conversion factor is retrieved from the `PHOTOM` reference file, and the pixel values in the science data are multiplied by this factor. The factor is also saved in the `PHOTMJSR` keyword within the header of the exposure file. The map of relative pixel areas is also appended to the exposure in a new extension called `AREA`. The average pixel area in units of steradians and square arcseconds is also saved in the science extension header, in the `PIXAR_SR` and `PIXAR_A2` keywords.


#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/photom/main.html) of the step.

#### Arguments

There are no optional arguments for this step

#### Reference files used

This step uses the [`PHOTOM` and `AREA`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/photom/reference_files.html) reference files. The `PHOTOM` reference file contains a table of conversion factors that depend on filter. The `AREA` reference file contains a map of the relative pixel areas across the detector.


#### Run the step

In [ ]:
for filename in flatfield_output_files:
    photom_step = PhotomStep()
    photom_step.output_dir = output_dir
    photom_step.save_results = True
    photom_step.run(filename)

Let's look at the new information that was added to the output file. 

In [ ]:
# When the output is saved, the photom step will
# attach a suffix of 'photomstep' to the input filename.
photom_output_files = sorted(glob(os.path.join(output_dir, '*_photomstep.fits')))

In [ ]:
# Open one of the output files and look at the contents
hdulist = fits.open(photom_output_files[0])

In [ ]:
hdulist.info()

In [ ]:
# The primary extension header is updated by the photom step
sci_header = hdulist['SCI'].header

And the mean pixel area:

In [ ]:
print('Mean pixel area in steradians: {}, and square arcseconds: {}'
      .format(sci_header['PIXAR_SR'], sci_header['PIXAR_A2']))

In [ ]:
# Let's pull out the science data and the newly-attached AREA extension
area_map = hdulist['AREA'].data
photom_science_data = hdulist['SCI'].data
hdulist.close()

Note the new `AREA` extension. Let's have a look:

In [ ]:
show_image(area_map, 0.95, 1.05, scale='linear', units='Relative Pixel Area')

In [ ]:
show_image(photom_science_data, 0.3, 5.0)

<a id='resample'> </a>
## The `Resample` step

#### Summary

This step resamples the calibrated slope image onto a distortion-free pixel grid. The output is a file with the suffix `_i2d.fits`. This file is for user-examination only. In the Stage 3 pipeline, the resample step will be called again when combining multiple images and creating the final, distortion-free mosaic image.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/main.html) of the step.

#### Arguments

There is a list of [optional Astrodrizzle-style](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/arguments.html) input parameters that can be used to customize the resampling process.

#### Reference files used

This step uses the [`DRIZPARS`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/reference_files.html) reference file. This file contains Astrodrizzle-style keywords that can be used to control the details of the resampling.

#### Run the step

Let's see what parameters are available:

In [ ]:
print(ResampleStep.spec)

In [ ]:
for filename in photom_output_files:
    resample_step = ResampleStep()
    resample_step.output_dir = output_dir
    resample_step.save_results = True
    resample_step.run(filename)

In [ ]:
# When the output is saved, the resample step will
# attach a suffix of 'resamplestep' to the input filename.
resample_output_files = sorted(glob(os.path.join(output_dir, '*_resamplestep.fits')))

In [ ]:
# Extract the data from the three resampled output files so we can look
# at the data
resample_data_0 = fits.getdata(resample_output_files[0])
resample_data_1 = fits.getdata(resample_output_files[1])
resample_data_2 = fits.getdata(resample_output_files[2])

The three images all look very similar, with the dithers between them being the most noticable differences.

In [ ]:
show_image(resample_data_0, 0.32, 1.0)

In [ ]:
show_image(resample_data_1, 0.32, 1.0)

In [ ]:
show_image(resample_data_2, 0.32, 1.0)

Now let's compare this to the data immediately prior to the resample step, in order to highlight the difference.

In [ ]:
show_image(photom_science_data, 0.32, 1.0)

Notice that the array size has changed:

In [ ]:
print(photom_science_data.shape)

In [ ]:
print(resample_data_0.shape)

This is the end of the Stage 2 pipeline for imaging. The outputs from this, along with a level 3 association file, can now be used as input to the Stage 3 pipeline, where they will be combined into a single mosaic image. This will be shown in the third notebook for this module.

[Top of notebook](#top)